# Block 1 — Mathematical Foundations of Tokenization, Batching & Attention Masking

This notebook builds the **mathematical backbone** for how LLMs go from raw text to tokens, how batches are formed and losses computed, and how attention masks (causal/padding/head) enforce the same principles architecturally.

---

## What you will learn in Block 1
1. **Objects & notation**: strings, normalization, tokenizer, vocabulary, special tokens.  
2. **Autoregressive factorization** of sequence probability.  
3. **Maximum-likelihood objective (NLL)** and its link to **cross-entropy**.  
4. **Batching with padding** and the **masked mean token loss**.  
5. **Perplexity** as exponential of mean NLL.  
6. **Attention masking (preview)**: causal mask, key-padding mask, and head masking.

---

## 1. Objects and notation

- **Alphabet & strings.**  
  Let $\Sigma$ be the raw character alphabet (e.g., Unicode code points).  
  Any finite string is an element:
  $$
  x \in \Sigma^{*}.
  $$

- **Normalization.**  
  A preprocessing map:
  $$
  \nu:\Sigma^{*} \to \Sigma^{*}, \qquad x \mapsto \nu(x),
  $$
  that may include:
  - Unicode normalization (e.g., NFKC),  
  - case folding (lowercasing),  
  - whitespace and punctuation rules.  

  *Remark:* Normalization **changes the support** of the data distribution by altering how strings map to tokens (e.g., “Café” → “Cafe”).

- **Tokenizer.**  
  A **tokenizer** is a function that segments normalized text into discrete units (tokens) from a fixed vocabulary $\mathcal{V}$.  
  Mathematically:
  $$
  \tau: \Sigma^{*} \to \mathcal{V}^{*}, \qquad
  \mathbf{t}=\tau(\nu(x))=(t_1,\dots,t_{L(x)}), \quad t_i \in \mathcal{V}.
  $$

  
  - Tokenizers define the *basic symbols* LLMs see.  
  - They can split text into **characters** (character-level), **words** (word-level), or **subwords** (BPE/Unigram).  
  - Subword tokenization (e.g., “tokenization” → `["token", "ization"]`) balances vocabulary size with sequence length.  
  - This mapping is deterministic: the same string always yields the same token sequence.  
  - Without tokenization, raw text would be unmanageable: the space of possible strings $\Sigma^{*}$ is infinite, but the space of tokens $\mathcal{V}$ is finite and tractable.

- **Vocabulary & special tokens.**  
  The vocabulary $\mathcal{V}=\{0,1,\dots,V-1\}$ contains learned tokens plus reserved IDs:
  - `<bos>` : beginning of sequence  
  - `<eos>` : end of sequence  
  - `<pad>` : padding (used in batching)  
  - `<unk>` : unknown token (for OOV strings)

- **Augmented sequence (optional).**  
  To mark boundaries explicitly:
  $$
  \tilde{\mathbf{t}} = (\texttt{<bos>},\, t_1,\dots,t_{L(x)},\, \texttt{<eos>})
  $$

- **Dataset.**  
  A dataset is a collection of $N$ independent samples:
  $$
  \mathcal{D} = \{\, x^{(n)} \,\}_{n=1}^{N}.
  $$
  After tokenization:
  $$
  \mathcal{D}_{\text{tok}} = \{\, \mathbf{t}^{(n)} \,\}_{n=1}^{N}, \qquad
  \mathbf{t}^{(n)} = (t^{(n)}_1, \dots, t^{(n)}_{L(x^{(n)})}).
  $$

---

## 2. Autoregressive factorization (preview)

An LLM with parameters $\theta$ assigns probability to a tokenized string via the **causal chain rule**:

$$
p_\theta(x) = \prod_{i=1}^{L(x)} p_\theta\!\big(t_i \mid t_{<i}\big),
\qquad t_{<i}=(t_1,\dots,t_{i-1}).
$$

With explicit boundaries:
$$
p_\theta(x) = \prod_{i=1}^{\tilde L} p_\theta\!\big(\tilde t_i \mid \tilde t_{<i}\big),
\quad \tilde t_1=\texttt{<bos>},\ \tilde t_{\tilde L}=\texttt{<eos>}.
$$

*Interpretation:*  
The model decomposes the text into **conditional next-token probabilities**; causality forbids peeking at the future.

---

## 3. Training objective (preview)

Given dataset $\mathcal{D}$, **maximum likelihood** maximizes the log-probability assigned to the data:

$$
\max_{\theta}\; \sum_{x \in \mathcal{D}} \sum_{i=1}^{L(x)} \log p_\theta\!\big(t_i \mid t_{<i}\big).
$$

Equivalently, we **minimize the negative log-likelihood (NLL)**.  
Later we will connect this to **softmax cross-entropy**, show the **masked mean over valid tokens** (excluding `<pad>`), and define **perplexity**:

$$
\mathrm{PPL} = \exp\!\big(\text{mean NLL per valid token}\big).
$$

---

## 4. Attention masking (preview)

Self-attention computes scaled dot products between queries and keys.  
To respect the mathematics above, we add:
- a **causal mask** (no attending to future positions),  
- a **key-padding mask** (ignore `<pad>` positions),  
- and optionally a **head mask** (enable/disable heads for ablation/interpretability).

Later we will show how these masks combine into the attention logits **before** the softmax.

---

### Outcome of Block 1

By the end of Block 1 you will be able to:
- Map raw strings $\to$ tokens with clear assumptions about normalization and special tokens.  
- Write and reason about $p_\theta(x)$ via autoregressive factorization.  
- Derive the MLE/NLL objective and relate it to cross-entropy.  
- Implement padding-aware **masked mean** losses and compute **perplexity**.  
- Explain how **attention masks** (causal/padding/head) enforce the same constraints architecturally.


## 2) Autoregressive Factorization and Training Objective

Now that we understand what **tokens** are and how a **tokenizer** converts text into discrete units, we ask:  
**How does a Large Language Model assign a probability to a whole sequence of tokens?**

---

### 2.1 The chain rule of probability

From basic probability theory:  
For any sequence of random variables $(X_1, X_2, \dots, X_L)$ we can always write:

$$
p(X_1, X_2, \dots, X_L) = \prod_{i=1}^L p(X_i \mid X_1, \dots, X_{i-1}).
$$

This is called the **chain rule**.  
It is an identity, not an assumption — it always holds.

- $p(X_1)$ is the probability of the first element.  
- $p(X_2 \mid X_1)$ is the probability of the second given the first.  
- $p(X_3 \mid X_1, X_2)$ is the probability of the third given the first two.  
- And so on.

---

### 2.2 Applying the chain rule to tokens

For a token sequence $\mathbf{t} = (t_1, t_2, \dots, t_L)$ produced by the tokenizer:

$$
p_\theta(\mathbf{t}) = \prod_{i=1}^L p_\theta(t_i \mid t_1, \dots, t_{i-1}),
$$

where $\theta$ are the model parameters (the weights of the neural network).

- $t_i$ is the $i$-th token.  
- $t_{<i} = (t_1, \dots, t_{i-1})$ is the prefix (all tokens before $i$).  
- Each factor is the probability of the **next token** given all previous ones.

With explicit boundary tokens `<bos>` (begin) and `<eos>` (end):

$$
p_\theta(\tilde{\mathbf{t}}) = \prod_{i=1}^{\tilde L} p_\theta(\tilde t_i \mid \tilde t_{<i}),
$$

where $\tilde L = L + 2$ because of the added boundaries.

---

### 2.3 Why autoregression?

- **Causality.** Humans generate text left-to-right; the model imitates this by only looking at the past.  
- **Simplicity.** Instead of modeling an entire sequence directly, we only need to model *next-token prediction*.  
- **Flexibility.** With this formulation we can generate text token by token: sample $t_1$ from $p(t_1)$, then $t_2$ from $p(t_2\mid t_1)$, and so on.

---

### 2.4 Training objective: Maximum Likelihood Estimation (MLE)

We want our model to assign **high probability** to real text sequences from the dataset.  

Given a dataset $\mathcal{D} = \{ \mathbf{t}^{(n)} \}_{n=1}^N$, the **log-likelihood** is:

$$
\log p_\theta(\mathcal{D}) = \sum_{n=1}^N \log p_\theta(\mathbf{t}^{(n)}).
$$

Using the autoregressive factorization:

$$
\log p_\theta(\mathbf{t}^{(n)}) = \sum_{i=1}^{L(x^{(n)})} \log p_\theta(t^{(n)}_i \mid t^{(n)}_{<i}).
$$

Thus the training objective is:

$$
\max_\theta \; \sum_{n=1}^N \sum_{i=1}^{L(x^{(n)})} \log p_\theta\!\big(t^{(n)}_i \mid t^{(n)}_{<i}\big).
$$

This is called **Maximum Likelihood Estimation (MLE).**

---

### 2.5 Why logs?

If we worked directly with the product $\prod p(t_i \mid t_{<i})$, probabilities quickly become astronomically small (multiplying many numbers less than 1).  
Taking the logarithm:

- Turns products into sums (easier to compute).  
- Stabilizes numerics (avoids underflow).  
- Matches information-theoretic interpretations (log-loss).  

---

### 2.6 Negative log-likelihood (NLL)

Instead of maximizing log-likelihood, we minimize the **negative log-likelihood**:

$$
\mathcal{L}_{\text{NLL}}(\theta) = -\sum_{n=1}^N \sum_{i=1}^{L(x^{(n)})} \log p_\theta(t^{(n)}_i \mid t^{(n)}_{<i}).
$$

This is the **loss function** of LLMs.  
Minimizing it means: *the model gets penalized whenever it assigns low probability to the correct next token.*

---

### 2.7 Connection to cross-entropy

At each position $i$, the model outputs a probability distribution over the vocabulary $\mathcal{V}$.  

If the true token is $t_i$, the per-token loss is:

$$
\ell_i = -\log p_\theta(t_i \mid t_{<i}).
$$

This is exactly the **cross-entropy** between the predicted distribution $p_\theta(\cdot \mid t_{<i})$ and the true one-hot distribution $y$ where $y_{k} = 1$ if $k=t_i$:

$$
\ell_i = H(y, p_\theta) = - \sum_{k=1}^{V} y_k \log p_\theta(k \mid t_{<i}).
$$

---

### 2.8 Gradient identity (intuition for learning)

The derivative of the cross-entropy with respect to the logits $z_{i,k}$ is:

$$
\frac{\partial \ell_i}{\partial z_{i,k}} = p_\theta(k \mid t_{<i}) - y_k.
$$

This means:
- If the model assigns too much probability to the wrong token, the gradient pushes it down.  
- If the model assigns too little to the correct token, the gradient pushes it up.  
- Learning is simply **adjusting logits so that predicted probabilities match observed tokens.**

---

### 2.9 Intuition

- Training an LLM = **teaching it to be a good next-token predictor**.  
- By predicting the next token well across billions of examples, the model learns grammar, semantics, style, reasoning patterns, and world knowledge.  
- All of this comes from one simple principle: *minimize negative log-likelihood (maximize the probability of real text).*

---

**Next (Block 1.3):**  
We extend this objective to **batches of sequences** (with different lengths), introduce **padding**, and show how to compute a **masked mean token loss** that ignores `<pad>` tokens.


## 3) Batching, Padding, and the Masked Token Loss

So far we looked at single sequences. But in practice, models are trained with **batches** of sequences in parallel, for efficiency.

---

### 3.1 Why batching?

- GPUs/TPUs can process many examples at once.  
- Instead of updating weights after every single sequence, we group **B sequences** into a **batch**.  
- This allows **vectorized operations** and faster convergence (by averaging gradients).

Formally:  
A batch $\mathcal{B}$ is a set of $B$ token sequences:
$$
\mathcal{B} = \{\mathbf{t}^{(1)}, \mathbf{t}^{(2)}, \dots, \mathbf{t}^{(B)}\}.
$$

---

### 3.2 The problem of variable lengths

Different sequences have different lengths:
- Example: `"The cat"` → 2 tokens.  
- Example: `"A very long sentence here"` → 6 tokens.  

Neural networks expect tensors with the **same shape**, so we cannot stack them directly.  
Solution: **Padding**.

---

### 3.3 Padding sequences

Let $L_{\max}$ be the length of the longest sequence in the batch.  
We build a matrix:

$$
T \in \mathbb{N}^{B \times L_{\max}}, \quad
T_{b,i} =
\begin{cases}
t^{(b)}_i & \text{if } i \leq L(\mathbf{t}^{(b)}), \\\\
\texttt{<pad>} & \text{otherwise}.
\end{cases}
$$

- Each row = one sequence.  
- Shorter sequences are padded with a special `<pad>` token until they reach length $L_{\max}$.

---

### 3.4 Mask of valid tokens

To ensure padding does not affect the loss, we build a **mask**:

$$
M \in \{0,1\}^{B \times L_{\max}}, \qquad
M_{b,i} =
\begin{cases}
1 & \text{if } T_{b,i} \neq \texttt{<pad>} \quad (\text{valid token}), \\\\
0 & \text{if } T_{b,i} = \texttt{<pad>} \quad (\text{ignored}).
\end{cases}
$$

---

### 3.5 Masked mean token loss

Without masking, the loss would include `<pad>` tokens, corrupting training.  
Instead we average only over **valid tokens**:

$$
\mathcal{L}_{\text{token}} =
- \frac{1}{N_{\text{valid}}}
\sum_{b=1}^B \sum_{i=1}^{L_{\max}}
M_{b,i} \; \log p_\theta\!\big(T_{b,i} \mid T_{b,<i}\big),
$$

where
$$
N_{\text{valid}} = \sum_{b=1}^B \sum_{i=1}^{L_{\max}} M_{b,i}.
$$

This ensures:
- Loss is **independent of how much padding there is**.  
- Gradients reflect only *real tokens*.  

---

### 3.6 Implementation intuition

Most deep learning frameworks implement this using:
- A **CrossEntropyLoss** with `ignore_index=pad_id` (in PyTorch).  
- An **attention_mask** to indicate valid tokens (in Transformers).

Thus:
- Forward pass uses the padded tensor.  
- Loss computation ignores `<pad>`.  
- Attention layers also ignore `<pad>` (via masking).



In [5]:
import torch
import torch.nn as nn

# Suppose we have 3 sequences of different lengths
pad_id = 0
batch = torch.tensor([
    [5, 6, 7, 8, pad_id, pad_id],      # length 4
    [9, 10, 11, pad_id, pad_id, pad_id], # length 3
    [12, 13, 14, 15, 16, 17]            # length 6 (max)
])

print("Batch shape:", batch.shape)  # (3, 6)

# Mask: 1 for valid tokens, 0 for pad
mask = (batch != pad_id).long()
print("Mask:\n", mask)

# Example: logits from a model (random for demo)
B, L, V = batch.shape[0], batch.shape[1], 20  # vocab size = 20
logits = torch.randn(B, L, V)  # model output

# Loss function that ignores pad tokens
loss_fn = nn.CrossEntropyLoss(ignore_index=pad_id)

# Shift targets by one position for next-token prediction
targets = batch.clone()
targets[targets == pad_id] = pad_id  # keep pad as pad
print("Targets:\n", targets)

# Flatten for CE loss
loss = loss_fn(logits.view(-1, V), targets.view(-1))
print("Masked loss:", loss.item())

ModuleNotFoundError: No module named 'torch'

## 4) Perplexity: measuring how well a model predicts

Up to now we focused on the **loss** (negative log-likelihood). But in language modeling, people often report **perplexity (PPL)**.

---

### 4.1 Definition

For a dataset of $N_{\text{valid}}$ valid tokens, perplexity is:

$$
\mathrm{PPL} = \exp\!\Bigg( \frac{1}{N_{\text{valid}}}
\sum_{n=1}^N \sum_{i=1}^{L(x^{(n)})}
-\log p_\theta(t^{(n)}_i \mid t^{(n)}_{<i}) \Bigg).
$$

This is simply the exponential of the **average negative log-likelihood per token**.

---

### 4.2 Intuition

- If $\mathrm{PPL}=1$: the model is perfect — it always assigns probability 1 to the correct token.  
- If $\mathrm{PPL}=V$ (vocabulary size): the model is as bad as random guessing.  
- Lower perplexity = better model.

In words: **perplexity measures “how many tokens the model is confused among, on average.”**

---

### 4.3 Why use perplexity?

- It is easier to interpret than raw log-loss.  
- It is standard in language modeling benchmarks (e.g., Penn Treebank, WikiText).  
- It allows fair comparison across models trained with different vocabularies (to some extent).

---

### 4.4 Example with code
Now let’s compute perplexity from the loss in PyTorch.


In [ ]:
import torch
import torch.nn as nn
import math

# Suppose we have predictions for a batch
pad_id = 0
targets = torch.tensor([
    [5, 6, 7, 8, pad_id, pad_id],
    [9, 10, 11, pad_id, pad_id, pad_id],
    [12, 13, 14, 15, 16, 17]
])

B, L = targets.shape
V = 20
logits = torch.randn(B, L, V)

# Loss function that ignores padding
loss_fn = nn.CrossEntropyLoss(ignore_index=pad_id, reduction="sum")

# Compute total loss (sum over tokens, not mean yet)
loss_sum = loss_fn(logits.view(-1, V), targets.view(-1))

# Count valid tokens (non-pad)
valid_tokens = (targets != pad_id).sum().item()

# Average NLL per token
nll_avg = loss_sum.item() / valid_tokens

# Perplexity
ppl = math.exp(nll_avg)

print("Average NLL per token:", nll_avg)
print("Perplexity:", ppl)


## 5) Attention Masking: Causal, Padding, and Head Masks

So far we focused on **token probabilities** and **loss functions**.  
Now we connect this theory to the **attention mechanism** inside Transformers — the core building block of LLMs.

---

### 5.1 Self-attention in a nutshell

In self-attention, every token can attend to every other token.  
For a sequence of length $L$ and embedding size $d$, we compute:

- **Queries:** $Q \in \mathbb{R}^{L \times d}$  
- **Keys:** $K \in \mathbb{R}^{L \times d}$  
- **Values:** $V \in \mathbb{R}^{L \times d}$  

The **attention scores** are:

$$
S = \frac{QK^\top}{\sqrt{d}} \quad \in \mathbb{R}^{L \times L}.
$$

After applying softmax row-wise, we get the **attention weights**:

$$
A = \text{softmax}(S),
$$

and then the output is:

$$
O = A \, V.
$$

---

### 5.2 The problem

- Without restrictions, a token at position $i$ could attend to **future tokens** $j>i$.  
- `<pad>` tokens could receive attention, contaminating representations.  
- Sometimes we want to **disable certain heads** for ablation or interpretability.

**Solution:** use **masks** that tell the model where it is allowed to attend.

---

### 5.3 Causal mask (autoregression)

**Goal:** prevent a token from looking into the future.

Definition:

$$
C \in \{0,1\}^{L \times L}, \qquad
C_{ij} =
\begin{cases}
1 & \text{if } j>i \quad (\text{future blocked}), \\\\
0 & \text{otherwise}.
\end{cases}
$$

Applied to scores:

$$
S \;\leftarrow\; S + (-10^9)\cdot C.
$$

This sets future positions to a very large negative value (≈$-\infty$), so after softmax they get probability 0.

---

### 5.4 Padding mask

**Goal:** ignore `<pad>` tokens added during batching.

Definition for a batch of size $B$:

$$
P \in \{0,1\}^{B \times L}, \qquad
P_{b,j} =
\begin{cases}
1 & \text{if token $j$ in sequence $b$ is <pad>}, \\\\
0 & \text{otherwise}.
\end{cases}
$$

When broadcasted into attention scores, positions marked with 1 are also masked out.

---

### 5.5 Head mask

**Goal:** enable or disable specific heads.

For $H$ attention heads:

$$
h \in \{0,1\}^H \quad \text{(or } h \in \{0,1\}^{B \times H}\text{ if per-example)}.
$$

- $h_h=0$ → that head is disabled (all its contributions zeroed).  
- Useful for **interpretability** (probing which heads matter), or for **structured pruning**.

---

### 5.6 Combined masking

For a full batch, the final scores are:

$$
S = \frac{QK^\top}{\sqrt{d}}.
$$

Then:

1. Broadcast $C$ (causal mask) to $(1,1,L,L)$.  
2. Broadcast $P$ (padding mask) to $(B,1,1,L)$.  
3. Combine:  
   $$
   M = \text{broadcast}(C) \;\lor\; \text{broadcast}(P).
   $$
4. Apply to scores:  
   $$
   S \;\leftarrow\; S + (-10^9)\cdot M.
   $$
5. Softmax row-wise over keys:  
   $$
   A = \text{softmax}(S).
   $$
6. Apply head mask (if any):  
   $$
   A \;\leftarrow\; A \odot \text{broadcast}(h).
   $$
7. Final output:  
   $$
   O = A \, V.
   $$

---

### 5.7 Why attention masking is essential

- **Causal mask:** enforces the autoregressive factorization (no cheating by looking at the future).  
- **Padding mask:** ensures that `<pad>` tokens do not leak into the computation.  
- **Head mask:** gives flexibility for analysis and pruning.

Together, these masks align the **mathematics of training (loss + padding)** with the **architecture of attention**.

---

**Next (Block 1.6):**  
We will implement these masks in **PyTorch code**:  
1. Build a causal mask matrix.  
2. Build a padding mask from a batch with `<pad>`.  
3. Show how they are applied inside a MultiheadAttention layer.  
4. Visualize the causal mask.


In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

# -------------------------------
# 1. Create toy batch (with padding)
# -------------------------------
pad_id = 0
batch = torch.tensor([
    [5, 6, 7, 8, pad_id, pad_id],      # length 4
    [9, 10, 11, pad_id, pad_id, pad_id], # length 3
    [12, 13, 14, 15, 16, 17]            # length 6 (max)
])

B, L = batch.shape
D = 16   # embedding dimension
H = 2    # number of heads

# -------------------------------
# 2. Build causal mask
# -------------------------------
# Shape: (L, L)
causal_mask = torch.triu(torch.ones(L, L) * float("-inf"), diagonal=1)

print("Causal mask:\n", causal_mask)

# -------------------------------
# 3. Build key-padding mask
# -------------------------------
# True where PAD tokens are present
padding_mask = (batch == pad_id)
print("Padding mask shape:", padding_mask.shape)  # (B, L)

# -------------------------------
# 4. MultiheadAttention with masks
# -------------------------------
attn = nn.MultiheadAttention(embed_dim=D, num_heads=H, batch_first=True)

# Random embeddings for the tokens
x = torch.randn(B, L, D)

# Apply attention with both masks
out, attn_weights = attn(
    x, x, x,
    attn_mask=causal_mask,         # (L, L)
    key_padding_mask=padding_mask  # (B, L)
)

print("Output shape:", out.shape)                 # (B, L, D)
print("Attention weights shape:", attn_weights.shape) # (B*H, L, L)

# -------------------------------
# 5. Visualize causal mask
# -------------------------------
plt.imshow(causal_mask.numpy(), cmap="gray")
plt.title("Causal Attention Mask (white = -inf)")
plt.colorbar()
plt.show()
